[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Vampire-Chan/SDUI/blob/main/ComfyUI.ipynb)


In [ ]:
#@title **Environment Setup**
#@markdown Let ComfyUI Update and setup automatically for you. Untick if you don't use Google Drive.

# Import necessary modules
import os  # Required for checking file paths
from IPython.display import clear_output  # Optional, to clear output after drive mounting

# User-configurable options with checkboxes for simple control
OPTIONS = {}

USE_GOOGLE_DRIVE = True  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}

# Store options in the dictionary
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

# Step 1: Mount Google Drive (if selected)
if OPTIONS['USE_GOOGLE_DRIVE']:
    print("\u2714 Mounting Google Drive...")  # Checkmark for success message

    %cd /
    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    CUSTOM_NODES_DIR = f"{WORKSPACE}/custom_nodes/ComfyUI-Manager"
    %cd /content/drive/MyDrive
else:
    WORKSPACE = 'ComfyUI'
    CUSTOM_NODES_DIR = f"{WORKSPACE}/custom_nodes/ComfyUI-Manager"

# Step 2: Clone the ComfyUI repository (if not already present)
if not os.path.exists(WORKSPACE):
    print("\u2714 Initial setup for ComfyUI...")
    !git clone https://github.com/comfyanonymous/ComfyUI $WORKSPACE
else:
    print("\u2714 ComfyUI folder already exists, skipping clone.")

%cd $WORKSPACE

# Step 3: Update the ComfyUI repository (if selected)
if OPTIONS['UPDATE_COMFY_UI']:
    print("\u2714 Updating ComfyUI...")
    !git pull
else:
    print("\u2714 Skipping ComfyUI update.")

# Step 4: Clone or Update the ComfyUI-Manager repository
if not os.path.exists(CUSTOM_NODES_DIR):
    print("\u2714 Cloning ComfyUI-Manager for the first time...")
    !git clone https://github.com/ltdrdata/ComfyUI-Manager.git $CUSTOM_NODES_DIR
else:
    %cd $CUSTOM_NODES_DIR
    if OPTIONS['UPDATE_COMFY_UI']:
        print("\u2714 Updating ComfyUI-Manager...")
        !git pull
    else:
        print("\u2714 Skipping ComfyUI-Manager update.")
    %cd $WORKSPACE

# Step 5: Install required dependencies
print("\u2714 Installing dependencies...")
!pip install xformers!=0.0.18 -r requirements.txt \
    --extra-index-url https://download.pytorch.org/whl/cu121 \
    --extra-index-url https://download.pytorch.org/whl/cu118 \
    --extra-index-url https://download.pytorch.org/whl/cu117

print("\u2714 Environment setup complete.")


In [ ]:
import os
import requests
import shutil
import urllib.parse
import time

# 🔥 Custom Colab Form Inputs
MODEL_URL = "https://civitai.com/api/download/models/1028683?type=Model&format=SafeTensor&size=pruned&fp=fp16"  #@param {type:"string"}
API_KEY = ""  #@param {type:"string"}
MODEL_TYPE = "checkpoints"  #@param ["checkpoints", "clip_vision", "vae", "loras", "controlnet", "style_models", "upscale_models"]
SAVE_TO_DRIVE = True  #@param {type:"boolean"}

def setup_environment():
    """Setup Google Drive if selected."""
    if SAVE_TO_DRIVE:
        print("\ud83d\udd17 Mounting Google Drive...")
        from google.colab import drive
        drive.mount('/content/drive')
        WORKSPACE = '/content/drive/MyDrive/ComfyUI'
    else:
        WORKSPACE = './ComfyUI'
    return WORKSPACE

# \ud83c\udf10 Start setup environment
WORKSPACE = setup_environment()

# Define the paths for model directories
MODEL_DIRS = {
    'checkpoints': 'models/checkpoints/',
    'clip_vision': 'models/clip_vision/',
    'vae': 'models/vae/',
    'loras': 'models/loras/',
    'controlnet': 'models/controlnet/',
    'style_models': 'models/style_models/',
    'upscale_models': 'models/upscale_models/'
}

# Update model paths to use Google Drive if SAVE_TO_DRIVE is True
MODEL_DIRS = {key: os.path.join(WORKSPACE, path) for key, path in MODEL_DIRS.items()}

# Create directories if they do not exist
for path in MODEL_DIRS.values():
    os.makedirs(path, exist_ok=True)

def download_file(url, save_path, api_key=None):
    """Download a file from a given URL and save it to the specified path, tracking size, progress, and speed."""
    try:
        headers = {}
        if api_key:
            headers['Authorization'] = f'Bearer {api_key}'

        response = requests.get(url, stream=True, allow_redirects=True, headers=headers)

        if response.status_code == 403 and not api_key:
            print("\u26a0\ufe0f Access denied. Retrying with API key...")
            return download_file(url, save_path, api_key=API_KEY)

        response.raise_for_status()

        filename = os.path.basename(urllib.parse.urlparse(url).path)
        if 'content-disposition' in response.headers:
            import cgi
            _, params = cgi.parse_header(response.headers['content-disposition'])
            filename = params.get('filename', filename)

        save_path = os.path.join(save_path, filename)

        total_size = int(response.headers.get('Content-Length', 0))
        downloaded = 0
        start_time = time.time()

        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
                    downloaded += len(chunk)
                    percent = (downloaded / total_size) * 100
                    elapsed_time = time.time() - start_time
                    speed = downloaded / (1024 * 1024 * elapsed_time)
                    print(f"\rDownloading {filename} - {percent:.2f}% - {speed:.2f} MB/s", end="")

        print(f"\n\u2705 Downloaded: {filename} to {save_path} - Total Size: {total_size / (1024 * 1024):.2f} MB")
    except requests.exceptions.RequestException as e:
        print(f"\u274c Error downloading {url}: {e}")

# \ud83c\udf10 Start model download
if MODEL_URL:
    print(f"\u23f3 Starting download for {MODEL_URL} to the '{MODEL_TYPE}' directory...")
    save_path = MODEL_DIRS.get(MODEL_TYPE, os.path.join(WORKSPACE, 'models/'))
    download_file(MODEL_URL, save_path, api_key=API_KEY)
else:
    print("\u274c No URL provided for download.")


In [ ]:
#@title **ComfyUI**
#@markdown Launch the ComfyUI and enjoy Stable Diffusion Generation on Colab. Make sure to UnTick if you use Colab without Google Drive. And also if ComfyUI fails to load and shows "module not found" then put the module name in the "required_modules" box in the format.

mount_gdrive = True #@param {type:"boolean"}

import subprocess
import threading
import time
import socket
import os
from google.colab import drive

# Function to mount Google Drive if the user selects it
def mount_google_drive():
    """Mount Google Drive to the Colab environment."""
    if mount_gdrive:
        print("\ud83d\udd27 Mounting Google Drive...")
        try:
            drive.mount('/content/drive')
            print("\u2705 Google Drive mounted successfully!")
        except Exception as e:
            print(f"\u274c Error mounting Google Drive: {e}")
            exit(1)
    else:
        print("\ud83d\udd27 Not mounting Google Drive, using local folder.")

# Function to start cloudflared download
def download_cloudflared(folder_path):
    """Download cloudflared to the specified folder with progress indication."""
    try:
        print("\u23f3 Downloading cloudflared...")
        url = "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb"
        output_path = os.path.join(folder_path, "cloudflared-linux-amd64.deb")
        result = os.system(f"wget --progress=bar:force {url} -O {output_path}")
        if result != 0:
            raise Exception("\u274c Error downloading cloudflared. Download failed.")
        print("\n\u2705 Download complete.")
        return output_path
    except Exception as e:
        print(str(e))
        exit(1)

# Function to install cloudflared
def install_cloudflared(deb_path):
    """Install cloudflared .deb file."""
    try:
        print("\ud83d\udd27 Installing cloudflared...")
        result = os.system(f"dpkg -i {deb_path}")
        if result != 0:
            raise Exception("\u274c Error installing cloudflared. Installation failed.")
        print("\u2705 Installation complete.")
    except Exception as e:
        print(str(e))
        exit(1)

# Function to check if CUDA is available
def check_cuda_availability():
    """Check if CUDA is available and print message."""
    import torch
    if torch.cuda.is_available():
        print("\u2705 CUDA is available, using GPU.")
    else:
        print("\u274c No CUDA device found. Running on CPU mode.")

# Function to monitor ComfyUI and launch cloudflared
def iframe_thread(port):
    """Monitor ComfyUI port and launch cloudflared when ready."""
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
    print("\nComfyUI finished loading, trying to launch cloudflared (if it gets stuck here cloudflared is having issues)\n")

    p = subprocess.Popen(["cloudflared", "tunnel", "--url", f"http://127.0.0.1:{port}"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            print("This is the URL to access ComfyUI:", l[l.find("http"):], end='')

# Set the folder path based on Google Drive mount option
folder_path = '/content/ComfyUI'  # Default folder path
mount_google_drive()  # Mount Google Drive if selected
if mount_gdrive:
    folder_path = '/content/drive/MyDrive/ComfyUI'  # Use Google Drive folder for ComfyUI

# Check for CUDA availability
check_cuda_availability()

# Automatically install missing modules if ModuleNotFoundError is raised
def install_missing_module(module_name):
    try:
        __import__(module_name)
    except ModuleNotFoundError:
        print(f"\u274c Module '{module_name}' not found. Installing it now...")
        os.system(f"pip install {module_name}")
        print(f"\u2705 Module '{module_name}' installed successfully.")

# List of critical modules to check and install if missing
required_modules = ['torch', 'flask', 'flask_cors', 'requests', 'torchsde', 'spandrel', 'kornia', 'piexif', 'segment_anything']  #@param

# Check and install missing modules
for module in required_modules:
    install_missing_module(module)

# Start cloudflared download and installation
deb_path = download_cloudflared(folder_path)
install_cloudflared(deb_path)

# Start the ComfyUI thread and monitor port
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

# Start ComfyUI from the selected folder path
!python {os.path.join(folder_path, 'main.py')} --dont-print-server


In [ ]:
#@title #Compress Archive
#@markdown Use this because Google Drive sometimes fails to read changes made through Google Colab.
#@markdown So it is better to zip and delete the folder contents.
from google.colab import drive
drive.mount('/content/drive')

import os
import shutil
import zipfile
import sys
import subprocess

# 🔥 Auto-install libraries if missing
def auto_install_libraries():
    """
    Automatically installs required libraries for 7z and rar support.
    """
    required_libraries = {
        'py7zr': 'py7zr',      # For .7z support
        'rarfile': 'rarfile',  # For .rar support
        'tqdm': 'tqdm'         # For progress bar
    }

    for module, package in required_libraries.items():
        try:
            __import__(module)
        except ImportError:
            print(f"📦 Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

    # Install 'unrar' for .rar support (if on Linux-based system)
    if sys.platform.startswith('linux'):
        try:
            subprocess.run(['apt-get', 'install', '-y', 'unrar'], check=True)
            print("✅ Installed 'unrar' binary successfully.")
        except subprocess.CalledProcessError:
            print("❌ Failed to install 'unrar'. Please install it manually using 'apt-get install unrar'.")

auto_install_libraries()

# Import newly installed libraries
import py7zr
import rarfile
from tqdm import tqdm  # For the progress bar

# 🔥 User Parameters
output_folder = "/content/drive/MyDrive/ComfyUI/output"  #@param {type: "string"}
archive_name = "output"  #@param {type: "string"}
archive_destination = "/content/drive/MyDrive/"  #@param {type: "string"}
file_type = "zip"  #@param ["zip", "7z", "rar"]
compression_level = "normal"  #@param ["store", "fast", "normal", "high", "maximum"]

# 🔥 Compression level mapping for zip and 7z
compression_map = {
    'store': 0,  # No compression
    'fast': 1,  # Fastest but less compressed
    'normal': 5,  # Default balance
    'high': 7,  # Higher compression
    'maximum': 9  # Maximum compression
}

def compress_and_clean_output(output_folder, archive_name, archive_destination, file_type, compression_level):
    """
    Compresses the contents of the output folder into a .zip, .7z, or .rar archive,
    moves the archive to the destination, and deletes the original contents of the output folder.

    Args:
      output_folder: Path to the folder to be compressed.
      archive_name: Name of the resulting archive file (without extension).
      archive_destination: Path to where the archive file should be saved.
      file_type: Type of the archive ('zip', '7z', 'rar').
      compression_level: Compression level ('store', 'fast', 'normal', 'high', 'maximum').
    """
    if not os.path.exists(output_folder):
        print(f"❌ Error: The folder '{output_folder}' does not exist.")
        return

    if not os.path.exists(archive_destination):
        print(f"❌ Error: The destination folder '{archive_destination}' does not exist.")
        return

    # Get the initial size of the output folder
    total_original_size = sum(
        os.path.getsize(os.path.join(root, file))
        for root, _, files in os.walk(output_folder)
        for file in files
    )

    # Set the file extension and path
    extension = {
        'zip': '.zip',
        '7z': '.7z',
        'rar': '.rar'
    }.get(file_type, '.zip')

    archive_path = os.path.join(archive_destination, f"{archive_name}{extension}")
    compression_level_value = compression_map.get(compression_level, 5)  # Default is 'normal'

    if file_type == 'zip':
        compression = zipfile.ZIP_STORED if compression_level == 'store' else zipfile.ZIP_DEFLATED
        with zipfile.ZipFile(archive_path, 'w', compression) as zipf:
            files = [os.path.join(root, file) for root, _, files in os.walk(output_folder) for file in files]
            for file_path in tqdm(files, desc="📦 Compressing as .zip", unit="file"):
                arcname = os.path.relpath(file_path, output_folder)
                zipf.write(file_path, arcname)

    elif file_type == '7z':
        with py7zr.SevenZipFile(archive_path, 'w', filters=[{'id': py7zr.FILTER_LZMA2, 'preset': compression_level_value}]) as archive:
            files = [os.path.join(root, file) for root, _, files in os.walk(output_folder) for file in files]
            for file_path in tqdm(files, desc="📦 Compressing as .7z", unit="file"):
                archive.write(file_path, os.path.relpath(file_path, output_folder))

    elif file_type == 'rar':
        try:
            print("📦 Compressing as .rar (This may take some time, progress bar not supported)")
            subprocess.run(['rar', 'a', '-m' + str(compression_level_value), archive_path, output_folder], check=True)
        except subprocess.CalledProcessError:
            print("❌ Failed to create .rar archive. Please ensure 'rar' is installed and available in your system PATH.")
    else:
        print(f"❌ Unsupported file type '{file_type}' selected.")
        return

    # Get the size of the compressed archive
    total_compressed_size = os.path.getsize(archive_path)

    # Calculate compression percentage
    if total_original_size > 0:
        compression_percentage = (1 - (total_compressed_size / total_original_size)) * 100
    else:
        compression_percentage = 0

    # Print compression statistics
    print(f"📦 Original size: {total_original_size / 1024**2:.2f} MB")
    print(f"📦 Compressed size: {total_compressed_size / 1024**2:.2f} MB")
    print(f"📦 Compression percentage: {compression_percentage:.2f}%")

    # Delete the contents of the output folder (but not the folder itself)
    for filename in os.listdir(output_folder):
        file_path = os.path.join(output_folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f'❌ Failed to delete {file_path}. Reason: {e}')

    print(f"✅ Successfully compressed and cleaned contents of {output_folder}")


# 🔥 Run the function with the provided parameters
compress_and_clean_output(output_folder, archive_name, archive_destination, file_type, compression_level)
